In [ ]:
import pandas as pd
import joblib

In [ ]:
multi_model = joblib.load("SIH_predict (2).pkl")
scaler_y = joblib.load("predict_scaler (2).pkl")
x_columns = joblib.load("x_columns (2).pkl")

In [ ]:
new_data = pd.DataFrame({
    'Amount (tons)': [150],
    'Transport Distance (km)': [200],
    'Material_Aluminium': [1],
    'Material_Copper': [0],
    'Route_Ore': [1],
    'Route_Recycled': [0],
    'Route_Both': [0],
    'Energy Source_Coal': [1],
    'Energy Source_Electricity': [0],
    'Energy Source_Both': [0]
})

In [ ]:
new_data = new_data.reindex(columns=x_columns, fill_value=0)

In [ ]:
def predict_cf_cs_gwp(data):
    pred_scaled = multi_model.predict(data)
    pred = scaler_y.inverse_transform(pred_scaled)
    return pred[0][0], pred[0][1], pred[0][2]

In [ ]:
carbon_fp, circularity_score, gwp = predict_cf_cs_gwp(new_data)

In [ ]:
recommendations = []

alt1 = new_data.copy()
alt1[['Route_Ore','Route_Recycled','Route_Both']] = [0,1,0]
cf_alt1 , cs_alt1 , gwp_alt1 = predict_cf_cs_gwp(alt1)
if cf_alt1 < carbon_fp:
    saving1 = ((carbon_fp - cf_alt1)/carbon_fp) * 100
    saving2 = ((gwp - gwp_alt1)/gwp) * 100
    recommendations.append(f"Switch to recycled to save {saving1:.1f}% CO₂ emissions and Global Warming Potential by {saving2:.2f}%")

alt2 = new_data.copy()
alt2[['Energy Source_Coal','Energy Source_Electricity','Energy Source_Both']] = [0,1,0]
cf_alt2, cs_alt2 ,gwp_alt2 = predict_cf_cs_gwp(alt2)
if cf_alt2 < carbon_fp:
    saving1 = ((carbon_fp - cf_alt2)/carbon_fp) * 100
    saving2 = ((gwp - gwp_alt2)/gwp) * 100
    recommendations.append(f"Switch to electricity to save {saving1:.1f}% CO₂ emissions and Global Warming Potential by {saving2:.1f}%")

alt3 = new_data.copy()
alt3['Transport Distance (km)'] = alt3['Transport Distance (km)'] * 0.5
cf_alt3, cs_alt3 , gwp_alt3 = predict_cf_cs_gwp(alt3)
if cf_alt3 < carbon_fp:
    saving1 = ((carbon_fp - cf_alt3)/carbon_fp) * 100
    saving2 = ((gwp - gwp_alt3)/gwp) * 100
    recommendations.append(f"Reduce transport distance by 50% to save {saving1:.1f}% CO₂ emissions and Global warming Potential by {saving2:.1f}%")

if cs_alt1 > circularity_score:
    gain = cs_alt1 - circularity_score
    recommendations.append(f"Switch to recycled to boost circularity score by +{gain:.1f}")


print("AI Recommendations: ")
for r in recommendations:
    print(r)

AI Recommendations: 
Switch to recycled to save 88.6% CO₂ emissions and Global Warming Potential by 90.39%
Switch to electricity to save 36.5% CO₂ emissions and Global Warming Potential by 9.9%
Switch to recycled to boost circularity score by +82.2
